<a href="https://colab.research.google.com/github/Sriram-6740/Sentiment-Analysis-Using-Machine-learning/blob/main/sentiment_analysis_major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#importing libraries
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score


In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords_list = nltk.corpus.stopwords.words('english')
stopwords_list
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
stopwords_list.remove('no')
stopwords_list.remove('not')
len(stopwords_list)

177

In [12]:
#creating dataframe from dataset
df = pd.read_csv('/content/drive/MyDrive/Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [29]:
df['text']

9868             @USAirways #flight2031 the flight to Hell
10138    @USAirways Collectively since Friday I've been...
6783     @JetBlue I did see that! Working on picking up...
2222                                     @united following
14229    @AmericanAir Stuck on the plane in Charlotte. ...
                               ...                        
5426     @SouthwestAir Any plans to implement a call-ba...
7482     @JetBlue thanks. I will use the extra time to ...
1716     @united flight delayed...again...every time. "...
10606    @USAirways DCA to RSW at 8:30am Cancelled Flig...
7838     @JetBlue my mom's baggage is lost, in fly 1557...
Name: text, Length: 14640, dtype: object

In [14]:
def normalizer(tweet):
      only_letters = re.sub("[^a-zA-Z]", " ", tweet)
      only_letters = only_letters.lower()
      only_letters = only_letters.split()
      filtered_result = [word for word in only_letters if word not in stopwords.words('english')]
      lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
      lemmas = ' '.join(lemmas)
      return lemmas

In [15]:
df = shuffle(df)
y = df['airline_sentiment']
x = df.text.apply(normalizer)

In [16]:
vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [17]:
train_x,val_x,train_y,val_y = train_test_split(x_vectorized, y)

In [18]:
regressor = LogisticRegression(multi_class='multinomial',solver='newton-cg')
model = regressor.fit(train_x,train_y)

In [19]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
gs_clf = GridSearchCV(model,params,n_jobs=1,cv=10)
gs_clf = gs_clf.fit(train_x,train_y)
model = gs_clf.best_estimator_

In [20]:
y_pred = model.predict(val_x)

_f1 = f1_score(val_y, y_pred, average='micro')
_confusion = confusion_matrix(val_y, y_pred)
__precision = precision_score(val_y, y_pred, average='micro')
_recall = recall_score(val_y, y_pred, average='micro')
_statistics = {'f1_score': _f1,
               'confusion_matrix': _confusion,
               'precision': __precision,
               'recall': _recall
              }

In [21]:
print(_statistics)

{'f1_score': 0.7797814207650273, 'confusion_matrix': array([[2037,  200,   75],
       [ 263,  454,   67],
       [ 109,   92,  363]]), 'precision': 0.7797814207650273, 'recall': 0.7797814207650273}


In [22]:
test_feature = vectorizer.transform(['movie is worst'])
model.predict(test_feature)

array(['negative'], dtype=object)

In [23]:
import pickle
pick1 = {'vectorizer': vectorizer,
         'model1': model
         }
pickle.dump(pick1, open('models'+".p", "wb"))

In [24]:
model = pickle.load(open('models.p','rb'))

In [25]:
#deploying using streamlit

!pip install streamlit

!pip install pyngrok==4.1.1
from pyngrok import ngrok

     |████████████████████████████████| 7.5MB 4.0MB/s 
     |████████████████████████████████| 163kB 38.3MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 112kB 36.6MB/s 
     |████████████████████████████████| 4.6MB 33.1MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
     |████████████████████████████████| 122kB 47.1MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=53999c7e36ad5ac11d558fadb776cc88950a10d43f2c95f655a3db80825dfe7d
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=64755a571b2c458bb5a6be4f42879a0ffaadfb4f0f4dfdfd25188af37b097845
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [26]:
%%writefile app.py
import streamlit as st
import pickle
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import os
import nltk
nltk.download('stopwords')
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score

st.title('SENTIMENT ANALYSIS')
text = st.text_input("Enter your sentence")

if text is not None:
  st.text(text)
def predict_res(text):
  model = pickle.load(open('models.p','rb'))
  text_feature = model['vectorizer'].transform([text])
  model1 = model['model1'].predict(text_feature)
  return model1
  

if st.button('PREDICT'):
  st.write('Result....')
    
    #test_feature = vectorizer.transform(['awesome boy'])
  st.title(predict_res(text))

Writing app.py


In [27]:
!nohup streamlit run app.py &

url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://9e99a103ab73.ngrok.io'